# Pushkin as Traveller

Pushkin grew up in Moscow and St. Petersburg. In 1819 he was exiled, at the age of nineteen, to a small town called Kishinev and then he was moved to his family estate in Ekaterinoslav. He remained in exile for nearly seven years until 1826 when Tsar Nicholas came to power. Below you can see a map of his travels throughout Russia.

<img src="https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/O.Kiprensky_-_A.S.Pushkin.jpg" alt="Portrait of A.S. Pushkin" style="width: 300px;"/>

[Original version](https://www.rgo.ru/ru/article/pushkin-puteshestvennik-linii-zhizni-linii-na-karte) of the graph that you can see below.

Also, see the [short journey through his biography](https://pushkinland.ru/2018/english/push1.php).

Some data was collected from other sources, such as:

- [https://en.wikipedia.org/wiki/Alexander_Pushkin](https://en.wikipedia.org/wiki/Alexander_Pushkin)
- [http://hronika.su/pushkin-a-s](http://hronika.su/pushkin-a-s)

In [6]:
import pandas as pd

from lets_plot import *

In [7]:
LetsPlot.setup_html()

In [8]:
def append_stay_time_calculator(places_df, moves_df):
    def stay_time_calculator(df):
        current_moves = moves_df[(moves_df["departure"] == df["name"])|(moves_df["arrival"] == df["name"])]
        years = sorted(current_moves["year"].unique())
        years_combined = []
        for year in years:
            if not any(years_combined) or years_combined[-1][1] + 1 != year:
                years_combined.append((year, year))
            else:
                years_combined[-1] = (years_combined[-1][0], year)
        years_combined = [str(year_from) if year_from == year_to else "{0}-{1}".format(year_from, year_to)
                          for year_from, year_to in years_combined]
        df["years"] = ", ".join(years_combined)
        return df

    places_df["years"] = None
    return places_df.apply(stay_time_calculator, axis=1)

In [9]:
raw_places_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/places.csv")
raw_moves_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/moves.csv")
moves_df = raw_moves_df.merge(raw_places_df, left_on="departure", right_on="name")\
             .rename(columns=dict(longitude="from_lon", latitude="from_lat"))\
             .drop(columns=["name"])\
             .merge(raw_places_df, left_on="arrival", right_on="name")\
             .rename(columns=dict(longitude="to_lon", latitude="to_lat"))\
             .drop(columns=["name"])
places_df = append_stay_time_calculator(raw_places_df, raw_moves_df)

In [10]:
ggplot() + \
    geom_livemap(const_size_zoomin=3) + \
    geom_curve(aes(x="from_lon", y="from_lat", xend="to_lon", yend="to_lat", color='path'),
               data=moves_df, curvature=-.1, spacer=4,
               tooltips=layer_tooltips().title("Trip name").line("@path"),
               arrow=arrow(type='closed', length=10, angle=15), size=1) + \
    geom_point(aes(x="longitude", y="latitude"), data=places_df, size=3, color='#F7F4F0',
               tooltips=layer_tooltips().title("@name").line("Visited in @years")) + \
    geom_point(aes(x="longitude", y="latitude"), data=places_df, size=1.7, color='black',
               tooltips='none') + \
    scale_color_manual(name="Trip name", values=['#addd8e', '#e34a33', '#8856a7', '#2c7fb8',
                                                 '#1c9099', '#006d2c', '#fec44f', '#636363']) + \
    coord_cartesian(xlim=[26, 58], ylim=[38, 62]) + \
    ggtitle("Alexander Pushkin's Trips") + \
    ggsize(800, 600) + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank',
          plot_title=element_text(size=20, margin=[20, 0, 0, 0]))